In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LeakyReLU, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [9]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
percent_val = 0.2

Konkatenacija podataka koji se posle koriste za adekvatnu raspodelu na skup za treniranje i validaciju

In [10]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [11]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [12]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

Ucitavanje stadardnih devijacija i strednih vrednosti

In [13]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [14]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Raspodela podataka kako bi se izbegla fiktivna stanja

In [15]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(len(y_command)):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*len(y_command)+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [17]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

In [16]:
time_steps = 10
input_shape = (time_steps,1)

In [23]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')
    
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 

In [24]:
val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

In [25]:
train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

# TESTIRANJE ARHITEKTURE i LR/BATCH

radjeno u vise iteracija sa razlicitim vrednostima zbog vremenske kompleksnosti

In [80]:
filter_size = [16,32,64]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
batch_size = [128,256]
#pool_type = [MaxPooling1D(pool_size = 3),MaxPooling1D(pool_size = 4), AveragePooling1D(pool_size = 3),AveragePooling1D(pool_size = 4)]
#global_pool_type = [GlobalAveragePooling1D(), GlobalMaxPool1D()]
time_steps_list = [8,10]

In [86]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
best_batch = None
#best_pool_type = None
#best_global_pool_type = None
best_step = None
for step in time_steps_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    
    for batch in batch_size:        
        for filt in filter_size:
            for kernel_1 in kernel_size_1:
                for kernel_2 in kernel_size_2:
                    for pool in pooling_size:
                        keras.utils.set_random_seed(7)
                        optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                        modelMSE = Sequential()
                        modelMSE.add(Input(shape=input_shape))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=32, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_2,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=64, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(GlobalAveragePooling1D())
                        modelMSE.add(Flatten())
                        modelMSE.add(Dense(128, activation=LeakyReLU()))
                        modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                        modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                        modelMSE.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2, shuffle = False)
                        test_predict_MSE = modelMSE.predict(val_x)



                        #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                        #radimo evaluaciju na mean 3 sigma testu
                        i =(len(y_command)-time_steps) - val_x.shape[0]%(len(y_command)-time_steps) -1
                        mean_3std = 0
                        for j in range(len(test_predict_MSE)):
                            if i%len(y_command) == 0:
                                i = step
                            if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                    mean_3std += 1
                            i += 1



                        if mean_3std > best_skor:
                            best_skor = mean_3std
                            best_filter = filt
                            best_kernel_1 = kernel_1
                            best_kernel_2 = kernel_2
                            best_pool = pool
                            best_batch = batch
                            #best_pool_type = ptype
                            #best_global_pool_type = gptype
                            best_step = step

                            print('best_skor: ', best_skor)
                            print('best_filter: ', best_filter)
                            print('best_kernel_1: ', best_kernel_1)
                            print('best_kernel_2: ', best_kernel_2)
                            print('best_batch', best_batch)
                            print('best_step', best_step)
                            print('best_pool', best_pool)
                            #print('best_pool_type', best_pool_type)
                            #print('best_global_pool_type',best_global_pool_type)
                                    
print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_batch', best_batch)
print('best_step', best_step)
print('best_pool', best_pool)
#print('best_pool_type', best_pool_type)
#print('best_global_pool_type',best_global_pool_type)

Epoch 1/10
9149/9149 - 24s - loss: 12.7065 - 24s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 23s - loss: 8.5592 - 23s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 23s - loss: 7.6903 - 23s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 23s - loss: 7.2565 - 23s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 23s - loss: 6.9664 - 23s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 23s - loss: 6.7475 - 23s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 23s - loss: 6.5673 - 23s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 23s - loss: 6.4109 - 23s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 23s - loss: 6.2799 - 23s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 23s - loss: 6.1725 - 23s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
best_skor:  289013
best_filter:  16
best_kernel_1:  3
best_kernel_2:  3
best_batch 128
best_step 8
best_pool 3
Epoch 1/10
9149/9149 - 25s - loss: 9.2969 - 25s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 23s - loss: 4.2920 - 23s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 24s - loss: 3.2638 - 24s/e

Epoch 4/10
9149/9149 - 26s - loss: 2.7532 - 26s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 26s - loss: 2.6427 - 26s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 26s - loss: 2.5620 - 26s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 27s - loss: 2.4997 - 27s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 2.4548 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 26s - loss: 2.4142 - 26s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 27s - loss: 2.3719 - 27s/epoch - 3ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
9149/9149 - 28s - loss: 7.5573 - 28s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 27s - loss: 3.3873 - 27s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 26s - loss: 2.7432 - 26s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 26s - loss: 2.5047 - 26s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 26s - loss: 2.3887 - 26s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 26s - loss: 2.3172 - 26s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 26s - loss: 2.2653 - 26s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - lo

Epoch 7/10
9149/9149 - 31s - loss: 2.2216 - 31s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 31s - loss: 2.1838 - 31s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 31s - loss: 2.1498 - 31s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 30s - loss: 2.1168 - 30s/epoch - 3ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
9149/9149 - 31s - loss: 6.1984 - 31s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 30s - loss: 3.0266 - 30s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 30s - loss: 2.7411 - 30s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 32s - loss: 2.5817 - 32s/epoch - 4ms/step
Epoch 5/10
9149/9149 - 34s - loss: 2.4619 - 34s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 31s - loss: 2.3585 - 31s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 30s - loss: 2.3105 - 30s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 30s - loss: 2.2633 - 30s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 30s - loss: 2.2217 - 30s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 30s - loss: 2.1887 - 30s/epoch - 3ms/step
9149/9149 [===================

4575/4575 - 19s - loss: 2.4020 - 19s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 19s - loss: 2.3351 - 19s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 19s - loss: 2.2944 - 19s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 19s - loss: 2.2580 - 19s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 19s - loss: 2.2288 - 19s/epoch - 4ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
4575/4575 - 20s - loss: 7.1445 - 20s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 19s - loss: 3.1553 - 19s/epoch - 4ms/step
Epoch 3/10
4575/4575 - 19s - loss: 2.6779 - 19s/epoch - 4ms/step
Epoch 4/10
4575/4575 - 19s - loss: 2.4673 - 19s/epoch - 4ms/step
Epoch 5/10
4575/4575 - 19s - loss: 2.3612 - 19s/epoch - 4ms/step
Epoch 6/10
4575/4575 - 20s - loss: 2.3026 - 20s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 20s - loss: 2.2530 - 20s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 19s - loss: 2.2135 - 19s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 20s - loss: 2.1855 - 20s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 20s - loss: 2.1590

9149/9149 - 26s - loss: 2.5389 - 26s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 2.4781 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 26s - loss: 2.4274 - 26s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 26s - loss: 2.3885 - 26s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
9149/9149 - 26s - loss: 11.1591 - 26s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 25s - loss: 8.1910 - 25s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 25s - loss: 7.4257 - 25s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 25s - loss: 6.9958 - 25s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 25s - loss: 6.7152 - 25s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 25s - loss: 6.5103 - 25s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 25s - loss: 6.3508 - 25s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 6.2210 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 27s - loss: 6.1158 - 27s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 25s - loss: 6.0228 - 25s/epoch - 3ms/step
9149/9149 [=============================

Epoch 10/10
9149/9149 - 30s - loss: 5.8292 - 30s/epoch - 3ms/step
9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
9149/9149 - 33s - loss: 6.5610 - 33s/epoch - 4ms/step
Epoch 2/10
9149/9149 - 32s - loss: 3.5120 - 32s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 32s - loss: 3.0017 - 32s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 32s - loss: 2.7104 - 32s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 31s - loss: 2.5205 - 31s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 32s - loss: 2.4153 - 32s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 32s - loss: 2.3298 - 32s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 32s - loss: 2.2613 - 32s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 32s - loss: 2.2046 - 32s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 32s - loss: 2.1629 - 32s/epoch - 3ms/step
9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
9149/9149 - 31s - loss: 6.7469 - 31s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 30s - loss: 3.6281 - 30s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 30s - loss: 3.

4575/4575 - 13s - loss: 3.4935 - 13s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 13s - loss: 3.2447 - 13s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 13s - loss: 3.0579 - 13s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 13s - loss: 2.9160 - 13s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 13s - loss: 2.8131 - 13s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 13s - loss: 2.7319 - 13s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
4575/4575 - 16s - loss: 9.7647 - 16s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 15s - loss: 4.3418 - 15s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 15s - loss: 3.6183 - 15s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 16s - loss: 3.2223 - 16s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 15s - loss: 2.9435 - 15s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 15s - loss: 2.7635 - 15s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 15s - loss: 2.6481 - 15s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 15s - loss: 2.5688 - 15s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 15s - loss: 2.5091 

## Trazenje jednostavnije arhitekture

In [18]:
filter_size_1 = [8,16]
filter_size_2 = [8,16]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
time_steps_list = [8,10]

In [19]:
best_skor = float('-inf')
best_filter = None
best_filter_2 = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
best_batch = None
#best_pool_type = None
#best_global_pool_type = None
best_step = None
for step in time_steps_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
         
    for filt in filter_size_1:
        for filt_2 in filter_size_2:
            for kernel_1 in kernel_size_1:
                for kernel_2 in kernel_size_2:
                    for pool in pooling_size:
                        keras.utils.set_random_seed(7)
                        optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                        modelMSE = Sequential()
                        modelMSE.add(Input(shape=input_shape))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=32, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Conv1D(filters=filt_2, kernel_size=kernel_2,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=64, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(GlobalAveragePooling1D())
                        modelMSE.add(Flatten())
                        modelMSE.add(Dense(128, activation=LeakyReLU()))
                        modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                        modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                        modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                        test_predict_MSE = modelMSE.predict(val_x)



                        #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                        #radimo evaluaciju na mean 3 sigma testu
                        i =(len(y_command)-time_steps) - val_x.shape[0]%(len(y_command)-time_steps) -1
                        mean_3std = 0
                        for j in range(len(test_predict_MSE)):
                            if i%len(y_command) == 0:
                                i = step
                            if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                    mean_3std += 1
                            i += 1



                        if mean_3std > best_skor:
                            best_skor = mean_3std
                            best_filter = filt
                            best_filter_2 = filt_2
                            best_kernel_1 = kernel_1
                            best_kernel_2 = kernel_2
                            best_pool = pool
                            #best_batch = batch
                            #best_pool_type = ptype
                            #best_global_pool_type = gptype
                            best_step = step

                            print('best_skor: ', best_skor)
                            print('best_filter: ', best_filter)
                            print('best_filter_2', best_filter_2)
                            print('best_kernel_1: ', best_kernel_1)
                            print('best_kernel_2: ', best_kernel_2)
                            #print('best_batch', best_batch)
                            print('best_step', best_step)
                            print('best_pool', best_pool)
                            #print('best_pool_type', best_pool_type)
                            #print('best_global_pool_type',best_global_pool_type)

print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_filter_2', best_filter_2)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
#print('best_batch', best_batch)
print('best_step', best_step)
print('best_pool', best_pool)
#print('best_pool_type', best_pool_type)
#print('best_global_pool_type',best_global_pool_type)



Epoch 1/10

4575/4575 - 13s - loss: 26.8957 - 13s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 11s - loss: 9.2078 - 11s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 11s - loss: 7.1795 - 11s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 11s - loss: 6.5880 - 11s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 11s - loss: 6.2561 - 11s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 11s - loss: 6.0229 - 11s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 10s - loss: 5.8380 - 10s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 11s - loss: 5.6800 - 11s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 11s - loss: 5.5400 - 11s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 11s - loss: 5.4177 - 11s/epoch - 2ms/step
9149/9149 [==============================] - 12s 1ms/step
best_skor:  277682
best_filter:  8
best_filter_2 8
best_kernel_1:  3
best_kernel_2:  3
best_step 8
best_pool 3
Epoch 1/10
4575/4575 - 13s - loss: 21.6901 - 13s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 11s - loss: 5.4244 - 11s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 11s - loss: 4.3310 - 1

Epoch 8/10
4575/4575 - 12s - loss: 5.2575 - 12s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 12s - loss: 5.1053 - 12s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 12s - loss: 4.9723 - 12s/epoch - 3ms/step
9149/9149 [==============================] - 12s 1ms/step
Epoch 1/10
4575/4575 - 12s - loss: 19.3988 - 12s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 10s - loss: 5.4328 - 10s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 12s - loss: 4.3882 - 12s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 12s - loss: 3.8157 - 12s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 12s - loss: 3.4536 - 12s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 12s - loss: 3.2134 - 12s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 12s - loss: 3.0447 - 12s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 12s - loss: 2.9129 - 12s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 12s - loss: 2.8007 - 12s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 12s - loss: 2.7056 - 12s/epoch - 3ms/step
9149/9149 [==============================] - 12s 1ms/step
Epoch 1/10
4575/4575 - 13s - loss: 1

9149/9149 [==============================] - 13s 1ms/step
Epoch 1/10
4575/4575 - 14s - loss: 15.5879 - 14s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 11s - loss: 3.9070 - 11s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 13s - loss: 3.1287 - 13s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 13s - loss: 2.7366 - 13s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 13s - loss: 2.5111 - 13s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 14s - loss: 2.3495 - 14s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 14s - loss: 2.2228 - 14s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 15s - loss: 2.1248 - 15s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 16s - loss: 2.0535 - 16s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 15s - loss: 1.9960 - 15s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
4575/4575 - 16s - loss: 16.9362 - 16s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 14s - loss: 3.6261 - 14s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 14s - loss: 2.9553 - 14s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 14s - loss: 2

Epoch 5/10
4575/4575 - 14s - loss: 6.6997 - 14s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 14s - loss: 6.3873 - 14s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 14s - loss: 6.1195 - 14s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 14s - loss: 5.8860 - 14s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 14s - loss: 5.6833 - 14s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 13s - loss: 5.4986 - 13s/epoch - 3ms/step
9149/9149 [==============================] - 13s 1ms/step
best_skor:  288068
best_filter:  8
best_filter_2 8
best_kernel_1:  3
best_kernel_2:  3
best_step 10
best_pool 4
Epoch 1/10
4575/4575 - 16s - loss: 17.9139 - 16s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 14s - loss: 5.6504 - 14s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 14s - loss: 4.3554 - 14s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 14s - loss: 3.6055 - 14s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 14s - loss: 3.1824 - 14s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 14s - loss: 2.9477 - 14s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 14s - loss: 2.7740 - 14s/

Epoch 8/10
4575/4575 - 12s - loss: 2.5950 - 12s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 12s - loss: 2.4899 - 12s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 12s - loss: 2.4062 - 12s/epoch - 3ms/step
9149/9149 [==============================] - 13s 1ms/step
Epoch 1/10
4575/4575 - 13s - loss: 14.9271 - 13s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 12s - loss: 4.7691 - 12s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 12s - loss: 3.6023 - 12s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 13s - loss: 3.1376 - 13s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 12s - loss: 2.8691 - 12s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 12s - loss: 2.6856 - 12s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 12s - loss: 2.5336 - 12s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 11s - loss: 2.4020 - 11s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 12s - loss: 2.2873 - 12s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 11s - loss: 2.1887 - 11s/epoch - 2ms/step
9149/9149 [==============================] - 12s 1ms/step
Epoch 1/10
4575/4575 - 13s - loss: 1

Epoch 3/10
4575/4575 - 15s - loss: 3.7566 - 15s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 15s - loss: 3.1474 - 15s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 15s - loss: 2.8075 - 15s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 15s - loss: 2.5881 - 15s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 15s - loss: 2.3955 - 15s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 14s - loss: 2.2308 - 14s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 13s - loss: 2.1145 - 13s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 14s - loss: 2.0269 - 14s/epoch - 3ms/step
9149/9149 [==============================] - 14s 1ms/step
Epoch 1/10
4575/4575 - 14s - loss: 17.1480 - 14s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 13s - loss: 7.1552 - 13s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 13s - loss: 6.1140 - 13s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 13s - loss: 5.5061 - 13s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 12s - loss: 5.0742 - 12s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 13s - loss: 4.7818 - 13s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 12s - l